In [1]:
import datasets

in_jsonl = '../2_brevity_benchmark/phi3_arena_winners.jsonl'
dataset = datasets.load_dataset('json', data_files=in_jsonl, split='train')
n_rows = len(dataset)
print(str(n_rows))

33000


In [2]:
# Function to filter out duplicates.
def filter_out_duplicates(example, seen_questions = set()):
    if example['question'] in seen_questions:
        return False
    else:
        seen_questions.add(example['question'])
        return True

dataset = dataset.filter(filter_out_duplicates)
len(dataset)

26968

In [3]:
# Function to filter out errors, normal answers, and 
# short answers that are not much shorter than normal answers.
def filter_out_bad_data(example):
    name_best, ans_best, ans_normal = example['name-best'], example['answer-best'], example['answer-normal']
    # Include only if:
    return name_best!='ERROR' and name_best=='short' and len(ans_best)<0.6*len(ans_normal)

dataset = dataset.filter(filter_out_bad_data)
len(dataset)

21394

In [4]:
# Rename columns to match DPO format:
# https://huggingface.co/docs/trl/dpo_trainer#expected-dataset-format
dataset = dataset.map(lambda row: {
    'question-id': row['question_id'],
    'prompt': row['question'],
    'chosen': row['answer-short'],
    'rejected': row['answer-normal'],
    })
dataset = dataset.remove_columns([
    'question_id', 'question', 'name-best', 'answer-best', 'answer-normal', 'answer-short'])
dataset[0]

Map:   0%|          | 0/21394 [00:00<?, ? examples/s]

{'question-id': '58210e39b3fd4441a2bd4a518bb44c2d',
 'prompt': 'What is the difference between OpenCL and CUDA?',
 'chosen': 'CUDA architecture and optimizations.',
 'rejected': 'OpenCL (Open Computing Language) and CUDA (Compute Unified Device Architecture) are both parallel computing platforms and programming models designed for general-purpose computing on GPUs (GPGPU). The main differences between them are:\n\n1. Platform and vendor: OpenCL is an open standard developed by the Khronos Group, which allows developers to write programs that can run on various hardware platforms, including GPUs from different vendors like NVIDIA, AMD, and Intel. CUDA, on the other hand, is a proprietary platform developed by NVIDIA, primarily targeting NVIDIA GPUs.\n\n2. Language: OpenCL uses a C-like language for writing kernels, which are functions executed on the GPU. CUDA also uses a C-like language, but it has additional features and extensions specific to NVIDIA GPUs.\n\n3. Performance: CUDA is g

In [5]:
out_jsonl = 'phi3-arena-short-dpo-train.jsonl'
dataset.to_json(out_jsonl, lines=True)

Creating json from Arrow format:   0%|          | 0/22 [00:00<?, ?ba/s]

18495200